# Midterm Solutions

## FINM 25000 - 2024

### UChicago Financial Mathematics

* Tobias Rodriguez del Pozo
* tobiasdelpozo@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is `115` points.
* You have `180` minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.

Submission
* You will upload your solution to the `Midterm` assignment on Canvas. (Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers,) and it should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_data.xlsx`

This file has sheets for...
* `info` - names of each stock ticker
* `excess returns` - weekly excess returns on several stocks
* `SPY` - weekly excess returns on SPY
* `forecasting` - monthly data on `USO` asset returns and two forecasting signals.

Note 
* the data for `excess returns` and `SPY` is **weekly** so any annualizations should use `52` weeks in a year.
* the data for `forecasting` is monthly, so any annualization should use `12` months in a year.

#### If useful
here is code to load in the data.

In [15]:
import pandas as pd
import numpy as np

In [16]:
FILEIN = '../data/midterm_data.xlsx'
sheet_exrets = 'excess returns'
sheet_spy = 'spy'

retsx = pd.read_excel(FILEIN, sheet_name=sheet_exrets, index_col=0, parse_dates=[0])
spy = pd.read_excel(FILEIN, sheet_name=sheet_spy, index_col=0, parse_dates=[0])

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 50     |
| 2       | 25     |
| 3       | 20     |
| 4       | 20     |

### Each numbered question is worth 5 points.

### Notation
(Hidden LaTeX commands)

$$\newcommand{\mux}{\tilde{\boldsymbol{\mu}}}$$
$$\newcommand{\wtan}{\boldsymbol{\text{w}}^{\text{tan}}}$$
$$\newcommand{\wtarg}{\boldsymbol{\text{w}}^{\text{port}}}$$
$$\newcommand{\mutarg}{\tilde{\boldsymbol{\mu}}^{\text{port}}}$$
$$\newcommand{\wEW}{\boldsymbol{\text{w}}^{\text{EW}}}$$
$$\newcommand{\wRP}{\boldsymbol{\text{w}}^{\text{RP}}}$$
$$\newcommand{\wREG}{\boldsymbol{\text{w}}^{\text{REG}}}$$
$$\newcommand{\targ}{\text{USO}}$$

# 1. Short Answer

### No Data Needed

These problem does not require any data file. Rather, analyze the situation conceptually, based on the information below. 

### 1

In what sense was ProShares `HDG` successful in hedging the `HFRI`, and in what sense was it unsuccessful in tracking the `HFRI`?

### 2

Did we find that **TIPS** have been useful in expanding the mean-variance frontier in the past? Did we conclude they might be useful in the future? Explain.

### 3.

Consider a Linear Factor Pricing Model (LFPM).

Which metric do we examine to understand its fit, (or errors)?

### 4.

What aspect of the classic mean-variance optimization approach leads to extreme answers? How did regularization help with this issue?

### 5.

Suppose investors are **not** mean-variance investors. If we find an investment with a Sharpe ratio higher than the "market", would this would be inconsistent with the CAPM?

### 6.

Which is more useful in assessing the model’s fit for pricing: the r-squared of the time-series regressions, the r-squared of the cross-sectional regression, or neither?

### 7.

GMO stated that they had a “contrarian” investment style. What did they mean by this? Was this seen in our investigation of the fund, GMWAX?

### 8.

How does Harvard make their portfolio allocation more realistic than a basic mean- variance optimization would imply? Is their approach easily implemented and computed from a numerical standpoint?

### 9.

If we want to hedge a portfolio's returns with respect to SPY, how could we calculate the optimal ratio? How would this ratio then be used to build the hedged position?

### 10.

Name one way in which Fama and French construct the factors that helps reduce cross- factor correlation.

***

# 2. Allocation


Consider a mean-variance optimization of **excess** returns provided in `midterm_data.xlsx.`

### 1. 

Report the following **annualized** statistics:
* mean
* volatility
* Sharpe ratio

In [17]:
import sys
sys.path.append('../cmds')
from utils import calc_risk_metrics, calc_return_metrics, calc_performance_metrics, calc_multivariate_regression, calc_mv_portfolio, calc_iterative_regression, calc_univariate_regression

metrics = calc_return_metrics(retsx, as_df=True, adj=52)
metrics

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Annualized Sortino Ratio
AAPL,0.319421,0.283883,1.125183,1.569426
MSFT,0.288087,0.240206,1.199334,1.705011
AMZN,0.239457,0.310389,0.771474,1.108573
NVDA,0.650658,0.468096,1.390011,2.371656
GOOGL,0.193328,0.274217,0.705020,1.120251
TSLA,0.569728,0.607026,0.938556,1.642329
XOM,0.124196,0.311613,0.398557,0.571626


### 2.

Report the weights of the tangency portfolio.

In [18]:
tan = calc_mv_portfolio(retsx.mean(), retsx.cov(), excess=True)
tan = pd.DataFrame(data=tan, columns=['Tan Portfolio'], index=retsx.columns)
tan

,Tan Portfolio
AAPL,0.322605
MSFT,0.787496
AMZN,-0.228607
NVDA,0.495996
GOOGL,-0.502721
TSLA,0.105975
XOM,0.019257


### 3.
Report the Sharpe ratio achieved by the tangency portfolio over this sample. Annualize it (accounting for weekly data.)

In [19]:
# Calculate the sharpe ratio of the tan portfolio
tan_rets = retsx.dot(tan)
calc_performance_metrics(tan_rets, adj=52).T

,Tan Portfolio
Annualized Return,0.563474
Annualized Volatility,0.358351
Annualized Sharpe Ratio,1.572409
Annualized Sortino Ratio,2.459251
Skewness,0.00438
Excess Kurtosis,1.899066
VaR (0.05),-0.063723
CVaR (0.05),-0.096174
Min,-0.223741
Max,0.193774


### 4.

* What weight is given to the asset with the lowest Sharpe ratio?
* What Sharpe ratio does the lowest (most negative) weight asset have?

Explain why the weights are not most extreme for the assets with the largest/smallest Share Ratios.

In [20]:
# Merge Sharpe ratio on tan portfolio
metrics = metrics.merge(tan, left_index=True, right_index=True)
metrics

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Annualized Sortino Ratio,Tan Portfolio
AAPL,0.319421,0.283883,1.125183,1.569426,0.322605
MSFT,0.288087,0.240206,1.199334,1.705011,0.787496
AMZN,0.239457,0.310389,0.771474,1.108573,-0.228607
NVDA,0.650658,0.468096,1.390011,2.371656,0.495996
GOOGL,0.193328,0.274217,0.705020,1.120251,-0.502721
TSLA,0.569728,0.607026,0.938556,1.642329,0.105975
XOM,0.124196,0.311613,0.398557,0.571626,0.019257


<font color=red>

Lowest sharpe (XOM) reseives a 0.01 weight, which is in fact the lowest but notably we are not short it. GOOG has the most negative weight, even though it doesn't have the worst sharpe (second worst). 

Diversification, diversification, diversification. Just because an asset has a mediocre SR, it might be uncorrelated to other assets. Therefore, it can still be a very valuable addition to our portfolio, and thus might have a higher tangency weight. For example, the highest Sharpe asset (NVDA), does not have the highest weight.

</font>

### 5.

To target a mean return of `0.001` weekly, would you be invested in the risk-free rate or borrowing from the risk-free rate?

In [21]:
np.sum(calc_mv_portfolio(retsx.mean(), retsx.cov(), excess=True, target=0.001))

0.09228463054305538

<font color='red'>

The weights sum up to 0.092. Therefore, we would be invested in the risk-free rate (at a proportion of 1 - 0.092 = 0.908).

</font>

***

# 3. Performance

### 1. 

Report the following performance metrics of excess returns for Tesla (`TSLA`).
* skewness
* kurtosis

You are not annualizing any of these stats.

What do these metrics indicate about the nature of the returns?

### 2. 

Report the maximum drawdown for `TSLA` over the sample.
* Ignore that your data is in excess returns rather than total returns.
* Simply proceed with the excess return data for this calculation.

### 3.

For `TSLA`, calculate the following metrics, relative to `SPY`:
* market beta
* alpha
* information ratio

Annualize alpha and information ratio.

*Recall that this is weekly data, with 52 weeks per year.*

### 4.

Comment on what you conclude about `TSLA` based on the statistics calculated in the previous question.

***

# 5. Forecasting

Forecast (total) returns on oil as tracked by the ETF ticker, `USO`. 

As signals, use two interest rate signals, as seen in Treasury-notes. (No need to consider anything specific about Treasury notes, just read these as macroeconomic signals.)
* T-note rate
* month-over-month change in the T-note rate

Find the all data needed for this problem in the sheet `forecasting`.

Note that the data in this sheet is monthly, not weekly.

In [22]:
forecasting = pd.read_excel(FILEIN, sheet_name='forecasting', index_col=0, parse_dates=[0])

### 1.

Estimate a forecasting regression of $\targ$ on the two (lagged) signals.

$$r_{t+1}^{\targ} = \alpha + \beta^{x}x_t + \beta^z z_t + \epsilon_{t+1}$$

where
* $x$ denotes the interest-rate signal.
* $z$ denotes the change in rate signal.

Report the r-squared, as well as the OLS estimates for the intercept and the two betas. (No need to annualize the stats.)

In [23]:
USO = forecasting.loc[:,['USO']]
signal = ['Tnote rate','Tnote rate change']
factors = forecasting.loc[:,['Tnote rate','Tnote rate change']].shift(1)
signal_reg = calc_multivariate_regression(USO, factors)
signal_reg

,Alpha,R-Squared,Tnote rate Beta,Tnote rate change Beta,Information Ratio,Tracking Error
USO,0.241445,0.028137,-0.009569,0.074061,0.687085,0.351405


<font color='red'>
Note: with annualization the alpha would be 0.24144/12 =  0.02012
</font>

### 2.

Use your forecasted returns, $\hat{r}^{\targ}_{t+1}$ to build trading weights:

$$w_t = 0.50 + 50\;\hat{r}^{\targ}_{t+1}$$

(So the rule says to hold 50% in the ETF plus/minus 50x the forecast. Recall the forecast is a monthly percentage, so it is a small number.)

Calculate the return from implementing this strategy. Denote this as $r^x_t$.

Report the first and last 5 values.

In [24]:
forecasted_rets = (np.array(forecasting.shift(1).loc[:,['Tnote rate','Tnote rate change']]) 
                   @ np.array(signal_reg.loc[:,['Tnote rate Beta','Tnote rate change Beta']].T))

forecast_ret = (pd.DataFrame(forecasted_rets,columns = ['Forecasted Return'],index= forecasting.index)) 

forecast_ret['Forecasted Return'] = (forecast_ret['Forecasted Return'] + 
                                     float(signal_reg['Alpha'] / 12))*50 + 0.5

strat = pd.DataFrame(forecast_ret['Forecasted Return'] * forecasting.loc[:,['USO']]['USO'], 
                     columns=forecast_ret.columns, 
                     index=forecast_ret.index).dropna()

display(strat.head())

display(strat.tail())

,Forecasted Return
date,
2009-06-30,0.046697
2009-07-31,-0.001041
2009-08-31,0.005171
2009-09-30,-0.001909
2009-10-31,-0.036695


,Forecasted Return
date,
2023-07-31,0.053407
2023-08-31,0.003366
2023-09-30,0.003397
2023-10-31,-0.079126
2023-11-30,-0.009729


### 3.

Calculate the following (annualized) performance metrics for both the passive investment, $r^\targ$, as well as the strategy implemented in the previous problem, $r^x$.

* mean
* volatility
* max drawdown

**Remember to annualize mean and volatility for monthly data.** (No need to annualize max drawdown.)

In [25]:
strat = strat.merge(forecasting[['USO']], left_index = True, right_index = True)

calc_performance_metrics(strat[['Forecasted Return', 'USO']]).T

,Forecasted Return,USO
Annualized Return,0.168447,-0.022956
Annualized Volatility,0.3066,0.356455
Annualized Sharpe Ratio,0.549401,-0.0644
Annualized Sortino Ratio,0.867917,-0.077082
Skewness,0.799252,-1.137681
Excess Kurtosis,3.559352,6.016314
VaR (0.05),-0.11783,-0.162767
CVaR (0.05),-0.168903,-0.256975
Min,-0.275698,-0.554497
Max,0.401469,0.353556


### 4.

Comment on whether the active strategy (using forecasting), $r^z$ is an improvement on the passive strategy of just holding $\targ$ directly.

<font color='red'>

It does pretty well. The Sharpe ratio is much better, and it has a much higher mean return whilst having lowe volatility. It also performs better in max-drawdown.

</font>